## If you like this  kernel Greatly Appreciate if you can  UPVOTE .Thank you


# Karnataka Education - EDA using Bokeh Visualisation
An NGO organisation takes initiatives to improve primary education in  India and want to carry out this program in Karnataka. It wants to target districts that fall behind in areas such as 

- Education Infrastructure

- Education Awareness

- Demographic features

Identify such districts that could be targeted in its first phase.

The source data for this exercise is obtained from data.gov.in

## Goal :

The goal of this notebook was primarily to:

1.       Explain the data, define your target and come up with features that can be used for modelling.

2.       Create a model based on your features and come up with the list of target districts

3.       Detailed analysis to include all components such as 

        - Data fetch
        - Data cleansing
        - Exploratory data analysis
        - Summary and Data Visualization
        


# Import Libraries

In [ ]:
# Import Pandas for data manipulation using dataframes

import pandas as pd

#Import Numpy for statistical calculations

import numpy as np

# Import Warnings 

import warnings

warnings.filterwarnings('ignore')

# Import Bokeh Library for output

from bokeh.io import output_notebook

output_notebook()

from bokeh.models import ColumnDataSource

from bokeh.models import HoverTool

from bokeh.models import LinearInterpolator,CategoricalColorMapper

from bokeh.io import show

from bokeh.plotting import figure

from bokeh.palettes import Spectral8

## Data Fetching

In [ ]:
# Read the raw data from the csv file extracted from data.gov.in into a dataframe object

data = pd.read_csv('../input/Town-wise-education - Karnataka.csv')

## Exploratory Data Analysis

In [ ]:
data.info()

Let us have a quick glance of what the data looks like by observing the first and last few rows

In [ ]:
data.head()

In [ ]:
data.tail()

Lets us examine the shape of this dataset

In [ ]:
data.shape

This means that we have 812 dimensions(rows) and 46 features (columns) in this dataset.
Now let us explore the data types of the dataset

In [ ]:
data.info()

The above observation shows that there are categorical and numerical features in the dataset.Let us explore further...

Now let us find out how many unique categories are available from the above categorical features.
Let us examine if there are any nulls in the dataset

In [ ]:
data.isnull().sum()

Let us also look at the entire metrics including the inter quartile range,mean,standard deviation for all the features


In [ ]:
data.describe(include = 'all')

Now let us look in detail the categorical features.For this basically extract all the categorical features into a dataframe object

In [ ]:
categorical_features = data.select_dtypes(include=[np.object])
categorical_features.info()

#### Let us observe the unique categories for all the object varables

In [ ]:
for column_name in data.columns:
    if data[column_name].dtypes == 'object':
        data[column_name] = data[column_name].fillna(data[column_name].mode().iloc[0])
        unique_category = len(data[column_name].unique())
        print("Feature '{column_name}' has '{unique_category}' unique categories".format(column_name = column_name,
                                                                                         unique_category=unique_category))

So based on the above results it is evident that 'Table Name' and 'Total/Rural/Urban' categorical features are redundant in nature which can be eliminated as it has no significance.

We can also eliminate 'State Code' as we are dealing with only Karnataka.

From above observations we can conclude that we do not need to do any imputation as there are no missing values.

Before we jump into data visualisations and explore further let us observe the general metrics using the describe function

ok now let me explore more about the data in detail and come up with some basic observations

## Data Cleansing

Now let us drop some of the columns as discussed above which has no importance in our EDA such as 
- 'Table Name'
- 'State Code'
- 'Total/Rural/Urban'

In [ ]:
data.drop('Table Name',axis =1,inplace = True)

data.drop('State Code',axis =1,inplace = True)

data.drop('Total/ Rural/ Urban',axis =1,inplace = True)


Let us further get more insights of the data by observing first few records say for a district & town 

In [ ]:
data.head()

#### Few Data Observations:

The general observation observed in each of the district are as follows
- 29 unique records for age group for each town code
    - 'All Ages' category is a summation of all ages 
- All of the below 12 categories are depicted in the form of persons ,male and female where persons is summation of male and female
    - Illiterate
    - Literate
    - Educational Level - Literate without Educational Level
    - Educational Level - Below Primary 
    - Educational Level - Primary
    - Educational Level - Middle
    - Educational Level - Matric/Secondary
    - Educational Level - Higher Secondary/Intermediate Pre-University/Senior Secondary
    - Educational Level - Non-technical Diploma or Certificate Not Equal to Degree
    - Educational Level - Technical Diploma or Certificate Not Equal to Degree
    - Educational Level - Graduate & Above
    - Unclassified
Note : Also we have 'Total - Persons', 'Total - Male','Total - Female' which do not have any significance to our analysis as these are summation of all the above categories person/male/female wise for each of the above category

#### Current Focus :

 - Since our main focus of this exercise is to improve the 'Primary Education'. So the analysis going forward  as per my assumption that the following categories fall in the need of primary education and rest not
    - Illiterate 
    - Educational Level - Literate without Educational Level
    - Educational Level - Below Primary 
    - Educational Level - Primary
    - Unclassified
    
   Which means that the following features can be eliminated from the dataset
   
    Total - Persons                                                                              
    Total - Males                                                                                
    Total - Females                                                                              
    Literate - Persons                                                                           
    Literate - Males                                                                             
    Literate - Females                                                                           
    Educational Level - Middle Persons                                                           
    Educational Level - Middle Males                                                             
    Educational Level - Middle Females                                                           
    Educational Level - Matric/Secondary Persons                                                 
    Educational Level - Matric/Secondary Males                                                   
    Educational Level - Matric/Secondary Females                                                 
    Educational Level - Higher Secondary/Intermediate Pre-University/Senior Secondary Persons    
    Educational Level - Higher Secondary/Intermediate Pre-University/Senior Secondary Males      
    Educational Level - Higher Secondary/Intermediate Pre-University/Senior Secondary Females    
    Educational Level - Non-technical Diploma or Certificate Not Equal to Degree Persons         
    Educational Level - Non-technical Diploma or Certificate Not Equal to Degree Males           
    Educational Level - Non-technical Diploma or Certificate Not Equal to Degree Females         
    Educational Level - Technical Diploma or Certificate Not Equal to Degree Persons             
    Educational Level - Technical Diploma or Certificate Not Equal to Degree Males               
    Educational Level - Technical Diploma or Certificate Not Equal to Degree Females             
    Educational Level - Graduate & Above Persons                                                 
    Educational Level - Graduate & Above Males                                                   
    Educational Level - Graduate & Above Females 
    
 
 #### Key Note:

For each district code & town code we have the total of all age groups as 'All Ages' category in 'Age Group' feature .In my opinion it is irrelevant as we need to focus on age groups which need primary education.So lets go ahead and remove these rows in the dataset

In [ ]:
data = data[data['Age-Group'] != 'All ages']

Now let us look in detail the numerical features that need to be dropped as they do not contribute to the primary education.


In [ ]:
columns = [ 'Total - Persons',
           'Total - Males',
           'Total - Females',
           'Literate - Persons',
           'Literate - Males',
           'Literate - Females',
           'Educational Level - Middle Persons',
           'Educational Level - Middle Males',
           'Educational Level - Middle Females',
           'Educational Level - Matric/Secondary Persons',
           'Educational Level - Matric/Secondary Males',
           'Educational Level - Matric/Secondary Females',
           'Educational Level - Higher Secondary/Intermediate Pre-University/Senior Secondary Persons',
           'Educational Level - Higher Secondary/Intermediate Pre-University/Senior Secondary Males',
           'Educational Level - Higher Secondary/Intermediate Pre-University/Senior Secondary Females',
           'Educational Level - Non-technical Diploma or Certificate Not Equal to Degree Persons',
           'Educational Level - Non-technical Diploma or Certificate Not Equal to Degree Males',
           'Educational Level - Non-technical Diploma or Certificate Not Equal to Degree Females',
           'Educational Level - Technical Diploma or Certificate Not Equal to Degree Persons',
           'Educational Level - Technical Diploma or Certificate Not Equal to Degree Males',
           'Educational Level - Technical Diploma or Certificate Not Equal to Degree Females',
           'Educational Level - Graduate & Above Persons',
           'Educational Level - Graduate & Above Males',
           'Educational Level - Graduate & Above Females']                                                                            

data.drop(columns,axis =1,inplace = True)

## Exploratory Data Analysis:

In this section I am going to visualize data with respect to categories focused on improving 'Primary Education'.

- Illiterate 
- Educational Level - Literate without Educational Level
- Educational Level - Below Primary 
- Educational Level - Primary
- Unclassified

 I have used Bokeh interactive data visualisation library to visualise data .Please note that the mouse hover over function is enabled to see the data visualisations for each district based on the above mentioned categories depicted below .Also please note the size of the circle depicts the size of the feature .

The visualisations depict district and age wise data representations for each of the above mentioned groups including total count as well as male and female counts.

#### Illeiterate :

In this section you can observe the data visualisation of the following features with respect to district and age group 

- Illiterate - Persons
- Illiterate - Males
- Illiterate - Females

In [ ]:
source = ColumnDataSource(dict(
    x = data['District Code'],
    y = data['Illiterate - Persons'],
    area = data['Area Name'],
    illerate = data['Illiterate - Persons'],
    illerate_male = data['Illiterate - Males'],
    illerate_female = data['Illiterate - Females'],
    age = data['Age-Group']
)       
)

size_mapper = LinearInterpolator(
    x = [data['Illiterate - Persons'].min(),data['Illiterate - Persons'].max()],
    y = [2,100]
)

color_mapper = CategoricalColorMapper(
    factors = list(data['Area Name'].unique()),
    palette = Spectral8
)

PLOT_OPTS = dict(height = 800,width = 800,x_range = (1,30),y_range=(10,100000))

p = figure(title = 'Illiteracy District/Area Wise',
           toolbar_location = 'above',
           tools = [HoverTool(
               tooltips = [('Area ','@area'),
                           ('Illerate - Total Persons ','@illerate'),
                           ('Illerate - Total Males ','@illerate_male'),
                           ('Illerate - Total Females ','@illerate_female'),
                           ('Age Group ','@age'),
                        ],show_arrow = False)],
           x_axis_label = 'District Code',
           y_axis_label = 'No of Illiterates',
           **PLOT_OPTS)

p.circle(x='x',
         y='y', 
         size = {'field': 'illerate','transform':size_mapper},
         color = {'field': 'area','transform':color_mapper},
         alpha = 0.7,
         legend = 'area',
         source = source)
p.legend.location = (0,-50)
p.right.append(p.legend[0])
p.legend.border_line_color = None
show(p,notebook_handle=True)

#### Educational Level - Literate without Educational Level :

In this section you can observe the data visualisation of the following features with respect to district and age group 
- Educational Level - Literate without Educational Level - Persons
- Educational Level - Literate without Educational Level - Males
- Educational Level - Literate without Educational Level - Females

In [ ]:
source = ColumnDataSource(dict(
    x = data['District Code'],
    y = data['Educational Level - Literate without Educational Level Persons'],
    area = data['Area Name'],
    illerate = data['Educational Level - Literate without Educational Level Persons'],
    illerate_male = data['Educational Level - Literate without Educational Level Males'],
    illerate_female = data['Educational Level - Literate without Educational Level Females'],
    age = data['Age-Group']
)       
)

size_mapper = LinearInterpolator(
    x = [data['Educational Level - Literate without Educational Level Persons'].min(),
         data['Educational Level - Literate without Educational Level Persons'].max()],
    y = [2,100]
)

color_mapper = CategoricalColorMapper(
    factors = list(data['Area Name'].unique()),
    palette = Spectral8
)

PLOT_OPTS = dict(height = 800,width = 800,x_range = (1,30),y_range=(10,6000))

p = figure(title = 'Educational Level - Literate without Educational Level (District vs. Age Wise)',
           toolbar_location = 'above',
           tools = [HoverTool(
               tooltips = [('Area ','@area'),
                           ('Educational Level - Literate without Educational Level - Total Persons ','@illerate'),
                           ('Educational Level - Literate without Educational Level - Total Males ','@illerate_male'),
                           ('Educational Level - Literate without Educational Level - Total Females ','@illerate_female'),
                           ('Age Group ','@age'),
                        ],show_arrow = False)],
           x_axis_label = 'District Code',
           y_axis_label = 'No of Educational Level - Literate without Educational Level',
           **PLOT_OPTS)

p.circle(x='x',
         y='y', 
         size = {'field': 'illerate','transform':size_mapper},
         color = {'field': 'area','transform':color_mapper},
         alpha = 0.7,
         legend = 'area',
         source = source)
p.legend.location = (0,-50)
p.right.append(p.legend[0])
p.legend.border_line_color = None
show(p,notebook_handle=True)

#### Educational Level - Below Primary:

In this section you can observe the data visualisation of the following features with respect to district and age group 
- Educational Level - Below Primary - Persons
- Educational Level - Below Primary - Males
- Educational Level - Below Primary - Females

In [ ]:
source = ColumnDataSource(dict(
    x = data['District Code'],
    y = data['Educational Level - Below Primary Persons'],
    area = data['Area Name'],
    illerate = data['Educational Level - Below Primary Persons'],
    illerate_male = data['Educational Level - Below Primary Males'],
    illerate_female = data['Educational Level - Below Primary Females'],
    age = data['Age-Group']
)       
)

size_mapper = LinearInterpolator(
    x = [data['Educational Level - Below Primary Persons'].min(),
         data['Educational Level - Below Primary Persons'].max()],
    y = [2,50]
)

color_mapper = CategoricalColorMapper(
    factors = list(data['Area Name'].unique()),
    palette = Spectral8
)

PLOT_OPTS = dict(height = 800,width = 800,x_range = (1,30),y_range=(10,100000))

p = figure(title = 'Educational Level - Below Primary (District vs. Age Wise)',
           toolbar_location = 'above',
           tools = [HoverTool(
               tooltips = [('Area ','@area'),
                           ('Educational Level - Below Primary Total Persons ','@illerate'),
                            ('Educational Level - Below Primary Total Males ','@illerate_male'),
                           ('Educational Level -  Below Primary Total Females ','@illerate_female'),
                           ('Age Group ','@age'),
                        ],show_arrow = False)],
           x_axis_label = 'District Code',
           y_axis_label = 'No of Educational Level - Below Primary Persons',
           **PLOT_OPTS)

p.circle(x='x',
         y='y', 
         size = {'field': 'illerate','transform':size_mapper},
         color = {'field': 'area','transform':color_mapper},
         alpha = 0.7,
         legend = 'area',
         source = source)
p.legend.location = (0,-50)
p.right.append(p.legend[0])
p.legend.border_line_color = None
show(p,notebook_handle=True)

#### Educational Level - Primary:

In this section you can observe the data visualisation of the following features with respect to district and age group 
- Educational Level - Primary - Persons
- Educational Level - Primary - Males
- Educational Level - Primary - Females

In [ ]:
source = ColumnDataSource(dict(
    x = data['District Code'],
    y = data['Educational Level - Primary Persons'],
    area = data['Area Name'],
    illerate = data['Educational Level - Primary Persons'],
    illerate_male = data['Educational Level - Primary Males'],
    illerate_female = data['Educational Level - Primary Females'],
    age = data['Age-Group']
)       
)

size_mapper = LinearInterpolator(
    x = [data['Educational Level - Primary Persons'].min(),
         data['Educational Level - Primary Persons'].max()],
    y = [2,50]
)

color_mapper = CategoricalColorMapper(
    factors = list(data['Area Name'].unique()),
    palette = Spectral8
)

PLOT_OPTS = dict(height = 800,width = 800,x_range = (1,30),y_range=(10,100000))

p = figure(title = 'Educational Level - Primary (District vs. Age Wise)',
           toolbar_location = 'above',
           tools = [HoverTool(
               tooltips = [('Area ','@area'),
                           ('Educational Level -  Primary Total Persons ','@illerate'),
                           ('Educational Level -  Primary Total Male ','@illerate_male'),
                           ('Educational Level -  Primary Total Female ','@illerate_female'),
                           ('Age Group ','@age'),
                        ],show_arrow = False)],
           x_axis_label = 'District Code',
           y_axis_label = 'No of Educational Level - Primary Persons',
           **PLOT_OPTS)

p.circle(x='x',
         y='y', 
         size = {'field': 'illerate','transform':size_mapper},
         color = {'field': 'area','transform':color_mapper},
         alpha = 0.7,
         legend = 'area',
         source = source)
p.legend.location = (0,-50)
p.right.append(p.legend[0])
p.legend.border_line_color = None
show(p,notebook_handle=True)

#### Unclassified:

In this section you can observe the data visualisation of the following features with respect to district and age group 
- Unclassified - Persons
- Unclassified - Males
- Unclassified - Females

In [ ]:
source = ColumnDataSource(dict(
    x = data['District Code'],
    y = data['Unclassified - Persons'],
    area = data['Area Name'],
    illerate = data['Unclassified - Persons'],
    illerate_male = data['Unclassified - Males'],
    illerate_female = data['Unclassified - Females'],
    age = data['Age-Group']
)       
)

size_mapper = LinearInterpolator(
    x = [data['Unclassified - Persons'].min(),
         data['Unclassified - Persons'].max()],
    y = [1,100]
)

color_mapper = CategoricalColorMapper(
    factors = list(data['Area Name'].unique()),
    palette = Spectral8
)

PLOT_OPTS = dict(height = 800,width = 800,x_range = (1,30),y_range=(10,400))

p = figure(title = 'Unclassified -  (District vs. Age Wise)',
           toolbar_location = 'above',
           tools = [HoverTool(
               tooltips = [('Area ','@area'),
                           ('Unclassified - Total Persons ','@illerate'),
                           ('Unclassified - Total Males ','@illerate_male'),
                           ('Unclassified - Total Females ','@illerate_female'),
                           ('Age Group ','@age'),
                        ],show_arrow = False)],
           x_axis_label = 'District Code',
           y_axis_label = 'Unclassified - Persons',
           **PLOT_OPTS)

p.circle(x='x',
         y='y', 
         size = {'field': 'illerate','transform':size_mapper},
         color = {'field': 'area','transform':color_mapper},
         alpha = 0.7,
         legend = 'area',
         source = source)
p.legend.location = (0,-50)
p.right.append(p.legend[0])
p.legend.border_line_color = None

show(p,notebook_handle=True)

Now let us look at summary total count of all categories with respect to total persons,total males & total females for each of the current features of our focus as show below.We are going to create three new features for the same namely
- Total
- Total_Males
- Total_Females

In [ ]:
data['Total']=data['Illiterate - Persons']+data['Educational Level - Below Primary Persons']+data['Educational Level - Literate without Educational Level Persons']+data['Educational Level - Primary Persons']+data['Unclassified - Persons']
data['Total_Males']=data['Illiterate - Males']+data['Educational Level - Below Primary Males']+data['Educational Level - Literate without Educational Level Males']+data['Educational Level - Primary Males']+data['Unclassified - Males']
data['Total_Females']=data['Illiterate - Females']+data['Educational Level - Below Primary Females']+data['Educational Level - Literate without Educational Level Females']+data['Educational Level - Primary Females']+data['Unclassified - Females']


Now let us visualise with the above new features created to get a summary holistic view of the entire analysis.

#### Summary (Total):

In this section you can observe the data visualisation of the following features with respect to district and age group 
- Total 
- Total - Males
- Total - Females

In [ ]:
source = ColumnDataSource(dict(
    x = data['District Code'],
    y = data['Total'],
    area = data['Area Name'],
    illerate = data['Total'],
    illerate_male = data['Total_Males'],
    illerate_female = data['Total_Females'],
    age = data['Age-Group']
)       
)

size_mapper = LinearInterpolator(
    x = [data['Total'].min(),
         data['Total'].max()],
    y = [5,100]
)

color_mapper = CategoricalColorMapper(
    factors = list(data['Area Name'].unique()),
    palette = Spectral8
)

PLOT_OPTS = dict(height = 800,width = 800,x_range = (1,30),y_range=(10,120000))

p = figure(title = 'Summary -  (District vs. Age Wise)',
           toolbar_location = 'above',
           tools = [HoverTool(
               tooltips = [('Area ','@area'),
                           ('Summary','@illerate'),
                           ('Total Males ','@illerate_male'),
                           ('Total Females ','@illerate_female'),
                           ('Age Group ','@age'),
                        ],show_arrow = False)],
           x_axis_label = 'District Code',
           y_axis_label = 'Total Population needing Primary Education',
           **PLOT_OPTS)

p.circle(x='x',
         y='y', 
         size = {'field': 'illerate','transform':size_mapper},
         color = {'field': 'area','transform':color_mapper},
         alpha = 0.7,
         legend = 'area',
         source = source)
p.legend.location = (0,-50)
p.right.append(p.legend[0])
p.legend.border_line_color = None
show(p,notebook_handle=True)

## Conclusion:
    
- The above summary data visualisation depicts that the target districts that need to be focused with respect to primary education as part of the Phase 1 NGO initiative
    - Hubli Darwad 
    - Mysore
    - Bangalore
    - Belguam
    - Gulbarga
    - Bellary
    - Davanagiri
    - Mangalore
- Also it is observed that age groups of 0-6 years and 30-45 years need more attention for most of the cases
Scope of improvement :
To perform more detailed analysis and understand the reasons and come up with a predictive model.
Due to time constraints of doing this exercise this part is left for further exercise .


# If you like this  kernel Greatly Appreciate if you can  UPVOTE this kernel.